In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset

In [2]:
num_classes = 5
batchsize = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.001
num_epochs = 500

In [3]:
class EEGDataset(Dataset):
    def __init__(self, data_path, labels_path):
        self.data = np.load(data_path)
        self.labels = np.load(labels_path)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the EEG data and corresponding label
        eeg = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return eeg, label

# Example usage
train_dataset = EEGDataset("eeg_dataset/train_epochs.npy", "eeg_dataset/train_labels.npy")
val_dataset = EEGDataset("eeg_dataset/val_epochs.npy", "eeg_dataset/val_labels.npy")
test_dataset = EEGDataset("eeg_dataset/test_epochs.npy", "eeg_dataset/test_labels.npy")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

print(len(train_loader))
# Check one batch
for eeg_batch, label_batch in train_loader:
    print("EEG Batch Shape:", eeg_batch.shape)  # (batch_size, 14, 640)
    print("Label Batch Shape:", label_batch.shape)  # (batch_size,)
    break


65
EEG Batch Shape: torch.Size([16, 1, 640])
Label Batch Shape: torch.Size([16])


In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(64)
        
        
        self.l1 = nn.Linear(input_size, 256)
        self.l2 = nn.Linear(256, 64)
        self.l3 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.bn1(x)
        x = self.dropout(x)
        
        x = self.relu(self.l2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        
        x = self.l3(x)
        return x
        

In [5]:
model = MLP(640, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (eeg, labels) in enumerate(train_loader):
        eeg = eeg.view(-1, 640).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(eeg)
        loss = criterion(outputs, labels)
        
        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 16 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
            


epoch 1/500, step 16/65, loss = 1.7641
epoch 1/500, step 32/65, loss = 2.1736
epoch 1/500, step 48/65, loss = 1.8788
epoch 1/500, step 64/65, loss = 1.7093
epoch 2/500, step 16/65, loss = 1.6891
epoch 2/500, step 32/65, loss = 1.8388
epoch 2/500, step 48/65, loss = 1.7512
epoch 2/500, step 64/65, loss = 1.5289
epoch 3/500, step 16/65, loss = 1.6246
epoch 3/500, step 32/65, loss = 1.6621
epoch 3/500, step 48/65, loss = 1.7397
epoch 3/500, step 64/65, loss = 1.5319
epoch 4/500, step 16/65, loss = 1.5548
epoch 4/500, step 32/65, loss = 1.9507
epoch 4/500, step 48/65, loss = 1.7818
epoch 4/500, step 64/65, loss = 1.6632
epoch 5/500, step 16/65, loss = 1.8517
epoch 5/500, step 32/65, loss = 1.7734
epoch 5/500, step 48/65, loss = 1.7820
epoch 5/500, step 64/65, loss = 1.6719
epoch 6/500, step 16/65, loss = 1.5884
epoch 6/500, step 32/65, loss = 1.5251
epoch 6/500, step 48/65, loss = 1.6732
epoch 6/500, step 64/65, loss = 1.6427
epoch 7/500, step 16/65, loss = 1.9526
epoch 7/500, step 32/65, 

In [7]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for eeg, labels in test_loader:
        eeg = eeg.view(-1, 640).to(device)
        labels = labels.to(device)
        outputs = model(eeg)
        
        # # print(outputs)
        # print(torch.max(outputs, 1))
        # print("ACC")
        # print(labels)
        # break
        
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100 * (n_correct/n_samples)
    print(f'accuracy = {acc}')

accuracy = 18.83408071748879
